---

## 
- **Usage**: Optimized for automated workloads in production
- **Management**: ephemeral, automatically provisioned by the Databricks job scheduler when job is triggered
- **Termination**: Designed for a single purpose and terminate automatically and immediately after the job is completed
- **Cost efficiency**: cost less than all purpose and warehouse. $0.20 per DBU for classic and $0.45 for serveless
    https://www.databricks.com/product/pricing/lakeflow-jobs

---

In [0]:
def get_job_json(job_id:str):
  import requests
  import json
  token=dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
  headers = {
    "Authorization" : f"Bearer {token}",
    "Content-Type" : "application/json"
    }
  
  url = "https://experian-serasa-ecs-labs.cloud.databricks.com/api/2.2/jobs/get"

  body = {
    "job_id": int(job_id)
  }
  return requests.get(url,headers=headers,json=body).json()

In [0]:
def update_to_paused(job_id:str):
  import requests
  import json
  import unicodedata
  update_json = get_job_json(job_id)
  if update_json.get("message") is not None:
    return update_json.get("message")
  token=dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
  headers = {
    "Authorization" : f"Bearer {token}",
    "Content-Type" : "application/json"
    }
  url = "https://experian-serasa-ecs-labs.cloud.databricks.com/api/2.2/jobs/update"

  if update_json.get("settings").get("schedule") is not None:
    update_json["settings"]["schedule"]["pause_status"] = "PAUSED"

  payload_update = {"job_id": int(job_id) , "new_settings": update_json.get("settings") }
  struct_update = json.dumps(payload_update)
  r = requests.post(url, headers=headers, data=struct_update)
  return r.status_code

In [0]:
jobs_id = []
for job_id in jobs_ids:
  print(job_id,update_to_paused(job_id))